# WINDNODE ABW - Scenario Analysis

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Julian Endres" <br>

In [ ]:
######## WINDNODE ###########
# define and setup logger
from windnode_abw.tools.logger import setup_logger
logger = setup_logger()

# load configs
from windnode_abw.tools import config
config.load_config('config_data.cfg')
config.load_config('config_misc.cfg')

from windnode_abw.analysis import analysis
from windnode_abw.tools.draw import *
######## DATA ###########

import re
import pandas as pd

######## PostgreSQL ###########
import sys
import os
import getpass
from sqlalchemy import *

######## Plotting ###########

# Plotting
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

# set seaborn style
sns.set()

import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
import plotly.offline as pltly

In [ ]:
# specify what to import (in path ~/.windnode_abw/)
run_timestamp = '2020-06-17_125728_1month'

# select multiple scenarios manually or use 'ALL' to analyze all
# scenarios found in directory
#scenarios = [scenario, 'sq']
scenarios = ['ALL']

regions_scns, results_scns = analysis(run_timestamp=run_timestamp,
                   scenarios=scenarios)


In [ ]:
scenario = 'future'
time_range = '1 month'

In [ ]:
results_scns[scenario]['results_axlxt'].keys()

In [ ]:
TECH_NAMES = {
    'pv_roof_small': 'PV-roof Small',
    'pv_roof_large': 'PV-roof Large',
    'pv_ground':'PV-ground',
    'wind':'Wind',
}

df_rel_area = pd.DataFrame(
    index=results_scns[scenario]['results_axlxt']['Area required'].index)

for tech, area in results_scns[scenario]['results_axlxt']['Area required'].iteritems():
    df_rel_area[TECH_NAMES[tech]] = area.div(regions_scns[scenario].muns.area) * 100

In [ ]:
plot_kwds  = {
    'nrows': 2,
    'ncols': 2,
    'figsize': (10,10),
    'title': 'Rel. area used in %',
    'legend_label': '%',
    'cmap' :'viridis'
}

plot_geoplots(regions_scns[scenario],
              df_rel_area,
              plot_kwds)

In [ ]:
plot_kwds  = {
    'nrows': 2,
    'ncols': 2,
    'figsize': (10,10),
    'title': 'Area used',
    'legend_label': 'ha',
    'cmap' :'viridis'
}

region = regions_scns[scenario]
plot_geoplots(regions_scns[scenario],
              results_scns[scenario]['results_axlxt'],
              plot_kwds)

- missing
    - units
    - costs
    - emissions compared to SQ
    - emissions compared to 1990
    - Flächenbedarf per mun #31
        - Technologiespezisch!

## 2 Eigenversorgungsanteil

In [ ]:
autarky_ts = results_scns[scenario]['flows_txaxt']['Stromerzeugung'].drop(columns='import').sum(axis=1).unstack().div(
    results_scns[scenario]['flows_txaxt']['Stromnachfrage'].drop(columns='export').sum(axis=1).unstack()).stack()
autarky_ts = (autarky_ts>1).sum(level=1) / len(autarky_ts.index.get_level_values(level=0).unique())
autarky_ts = autarky_ts * 100

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax = autarky_ts.plot(kind='bar', ax=ax)
ax.set_xlabel('AGS')
ax.set_ylabel('%')
ax.set_title('rel. time of Autarky per month',
              fontsize=16,
              fontweight='normal')

#ax.set_yticks([20,50,100,300,500,1000])
#ax.yaxis.set_major_formatter(ScalarFormatter())

plt.show()

In [ ]:
autarky_ts = results_scns[scenario]['flows_txaxt']['Stromerzeugung'].drop(columns='import').sum(axis=1).unstack().div(
    results_scns[scenario]['flows_txaxt']['Stromnachfrage'].drop(columns='export').sum(axis=1).unstack()).stack()
autarky_ts = autarky_ts.unstack().join(autarky_ts.sum(level=0).rename('ABW total')).stack()

In [ ]:
px.line(autarky_ts.unstack(), log_y=True)

- Plotly lässt sich nicht als bereits berechnetes HTML anzeigen / nur zur veranschaulichung
- -> Binder

#### Darstellung mit matplotlib

- Summe über Gemeinde als TS zusätzlich
- relativer Zeitliche Anteil Autarky
- mehrere Plots für ags?
- Summen über Tage?

- geoplot

## Energieversorgungsmix Balken

In [ ]:
plot_balance_bar(regions_scns[scenario], results_scns[scenario]['flows_txaxt']['Stromerzeugung'].sum(level=1),
                results_scns[scenario]['flows_txaxt']['Stromnachfrage'].sum(level=1))

- irgendwas stimmt hier mit dem demand nicht?!
- kein gas?

## Energieversorgungsmix - Verlauf

In [ ]:
techs = {'hydro': 'Laufwasser',
         'bio': 'Bioenergie',
         'wind': 'Windenergie',
         'pv_ground': 'Photovoltaik (Freifläche)',
         'pv_roof_small': 'Photovoltaik (Aufdach <30 kW)',
         'pv_roof_large': 'Photovoltaik (Aufdach >30 kW)'
        }

In [ ]:
start = pd.to_datetime('2015-01-01 00:00:00')
end = pd.to_datetime('2015-02-20 00:00:00')

plot_timeseries(regions_scns[scenario], kind='el', ags=15001000, start=start, end=end)

In [ ]:
results_scns[scenario]['results_t'].keys()

In [ ]:
results_scns[scenario]['results_axlxt']['Stromerzeugung nach Gemeinde']

- Plotly lässt sich nicht als bereits berechnetes HTML anzeigen / nur zur veranschaulichung
- -> Binder

#### Darstellung mit matplotlib

- Summe über Gemeinde als TS zusätzlich
- relativer Zeitliche Anteil Autarky
- mehrere Plots für ags?
- Summen über Tage?

- geoplot

## Access Windnode Database at RLI via VPN

In [ ]:
def postgres_session():
    """SQLAlchemy session object with valid connection to reeem database"""
    
    print('Please provide connection parameters to database:\n' +
              'Hit [Enter] to take defaults')
    host = '192.168.11.220'
    port = '54321'
    database = 'windnode_abw'
    user = 'windnode' 
    password = getpass.getpass(prompt='password: ', stream=sys.stderr)
#     password = 'windnode'
    con = create_engine(
            'postgresql://' + '%s:%s@%s:%s/%s' % (user,
                                                  password,
                                                  host,
                                                  port,
                                                  database)).connect()
    print('Password correct! Database connection established.')
    return con

# start session
con = postgres_session()
# get metadata
meta = MetaData(con, schema="windnode")

In [ ]:
# list of tables
tables_windnode_db = [i for i in meta.tables.keys()]
tables_windnode_db

### Plot all feed in ts

In [ ]:
# Database select (SQL)
table_name ='windnode.wn_abw_feedints'
sql = text("""SELECT * FROM {}""".format(table_name))
df_feedin = pd.read_sql_query(sql, con)
df_feedin = df_feedin.set_index('timestamp')

In [ ]:
df_feedin.head()

In [ ]:
df_feedin.index.value_counts().value_counts()

In [ ]:
# Facts dict
info_dict = {}
#info_dict['Filename'] = ['{}_windnode_db'.format(pd.to_datetime('today').strftime("%Y-%m-%d"))]
info_dict['Value'] = ['Leistung P']
info_dict['Unit'] = ['MW']
info_dict['Y-Axis'] = ['{} in {}'.format(*info_dict['Value'], *info_dict['Unit'])]
info_dict['X-Axis'] = ['Stunde im Jahr']
info_dict['Title'] = ['Feed in timeseries']
info_dict['Metadata'] = [meta_str]

In [ ]:
data = [go.Scatter(x=df_feedin.index.tolist(), 
               y=df_feedin[col].tolist(), 
               name=col, line=dict(), mode='lines') for col in df_feedin]

In [ ]:
layout = go.Layout(
    title=''.join(info_dict['Title']),
    yaxis=dict(title=''.join(info_dict['Y-Axis'])),
    xaxis=dict(title=''.join(info_dict['X-Axis']))
)
fig = go.Figure(data=data, layout=layout)
pltly.iplot(fig)